Embedding 第一段階

In [10]:
import pandas as pd
import numpy as np
import os
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]

from embedding import get_embedding as g_e
import breakdown

In [3]:
#入力

talker = "Agent[03]"
sentence = "Agent[00]を占ったら黒だったよ。Agent[01]を占ったら白だったよ。"
brokendown = eval(breakdown.get_list(talker, sentence)) # outputがstrなことがあるから
[g_e(sentence) for sentence in brokendown]

[array([-0.05039302,  0.00036413,  0.01491035, ..., -0.01889357,
        -0.0138039 ,  0.04071731]),
 array([-0.05649778,  0.00434017,  0.02079348, ..., -0.02294088,
        -0.00889205,  0.03463059])]

In [4]:
brokendown

['Agent[00] is black when checked by Agent[03]',
 'Agent[01] is white when checked by Agent[03]']

In [16]:
#データセット準備

"""df = pd.read_csv("embedded.csv")←うまくいかない！"""


sentencedf = pd.DataFrame(
    {'text': brokendown, 'vector': [g_e(sentence) for sentence in brokendown]}
    )
print(sentencedf)

                                           text                                             vector
0  Agent[00] is black when checked by Agent[03]  [-0.05039301514625549, 0.0003641265502665192, ...
1  Agent[01] is white when checked by Agent[03]  [-0.05649777501821518, 0.004340166226029396, 0...


In [37]:
df = pd.read_csv("protocol_example.csv")
Agents = ["Agent[00]", "Agent[01]", "Agent[02]", "Agent[03]", "Agent[04]"]
Roles = ["VILLAGER", "SEER", "MEDIUM", "BODYGUARD", "WEREWOLF", "POSSESSED"]
Teams = ["BLACK", "WHITE"]

datalist = {}
for column in df.columns: # 
    ans = [] # あるカラムのすべての要素
    sentences = df[column].dropna().values # .dropna()でnp.nanは落とす
    for sentence in sentences:
        if "[AGENT]" in sentence and "[ROLE]" in sentence:
            for x in Agents:
                for y in Roles:
                    ans.append(sentence.replace('[AGENT]', x).replace('[ROLE]', y))
        elif "[AGENT]" in sentence and "[TEAM]" in sentence:
            for x in Agents:
                for y in Teams:
                    ans.append(sentence.replace('[AGENT]', x).replace('[ROLE]', y))
        elif "[AGENT]" in sentence:
            for x in Agents:
                ans.append(sentence.replace('[AGENT]', x).replace('[ROLE]', y))
        elif '[ROLE]' in sentence:
            for y in Roles:
                    ans.append(sentence.replace('[AGENT]', x).replace('[ROLE]', y))

    datalist[column] = ans
    

data = []
for key, val in (datalist.items()):
    for i in val:
        data.append([i, key])
        
Df_all = pd.DataFrame(data, columns=["sentence", "verb"])
from embedding import get_embedding
Df_all["embedded"] = Df_all["sentence"].map(lambda x: get_embedding(x))

In [39]:
# 一旦、indexを文章にしてdataframeに
ans = []
for column in df.columns:
    for sentence in df[column].dropna().values:
        ans.append([sentence, column])

Df = pd.DataFrame(ans, columns=["sentence", "verb"])

from embedding import get_embedding
Df["embedded"] = Df["sentence"].map(lambda x: get_embedding(x))

In [78]:
# コサイン類似度の計算
from embedding import cos_sim
sentencedf["most_similar_text"] = None
a=-1
for j in sentencedf['vector']:
    a=a+1
    similarities = []
    for i in Df['embedded']:
        try:
            similarities.append(cos_sim(i, j))
            
        except:
            similarities.append(0)
            print("error")
    #最も類似しているベクトルに対応するテキストの取得
    most_similar_indices = np.argmax(similarities, axis=0)
    print(most_similar_indices, a)
    
    
    sentencedf.loc[a, 'most_similar_text'] = Df.loc[most_similar_indices, 'sentence']

73 0
2 1


In [79]:
sentencedf

,text,vector,most_similar_text,most_similar_text2
0,Agent[00] is black when checked by Agent[03],"[-0.05039301514625549, 0.0003641265502665192, ...",[AGENT] must not be the [ROLE].,My choice is Agent[00].
1,Agent[01] is white when checked by Agent[03],"[-0.05649777501821518, 0.004340166226029396, 0...",[AGENT] must be the [ROLE].,My choice is Agent[01].


In [76]:
#試しに全体からembedding
sentencedf["most_similar_text2"] = None
a=-1
for j in sentencedf['vector']:
    a=a+1
    similarities = []
    for i in Df_all['embedded']:
        try:
            similarities.append(cos_sim(i, j))
            
            
        except:
            similarities.append(0)
            print("error")
    #最も類似しているベクトルに対応するテキストの取得
    most_similar_indices = np.argmax(similarities, axis=0)
    print(most_similar_indices, j)
    
    
    sentencedf.loc[a, 'most_similar_text2'] = Df_all.loc[most_similar_indices, 'sentence']

495 [-0.05039302  0.00036413  0.01491035 ... -0.01889357 -0.0138039
  0.04071731]
496 [-0.05649778  0.00434017  0.02079348 ... -0.02294088 -0.00889205
  0.03463059]
